## Vamos a extrer la información de los diferentes datasets de amazon 

In [597]:
import json
import pandas as pd
from glob import glob


def convert(x):
    ''' Convert a json string to a flat python dictionary
    which can be passed into Pandas. '''
    ob = json.loads(x)
    for k, v in ob.items():
        if isinstance(v, list):
            ob[k] = ','.join(str(v))
        elif isinstance(v, dict):
            for kk, vv in v.items():
                ob['%s_%s' % (k, kk)] = vv
            del ob[k]
    return ob

def convertAllJSON2CSV(): 
    for json_filename in glob('*.json'):
        csv_filename = '%s.csv' % json_filename[:-5]
        print 'Converting %s to %s' % (json_filename, csv_filename)
        df = pd.DataFrame([convert(line) for line in file(json_filename)])
        df.to_csv(csv_filename, encoding='utf-8', index=False)

In [4]:
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
overallDF = df[["overall","reviewText"]]

data = df.groupby("overall")["overall"].count()
plt.bar(data.index.values,data.values)

NameError: name 'df' is not defined

In [5]:
cachedStopWords

NameError: name 'cachedStopWords' is not defined

In [36]:
# Hay que igualar y coger muestras de tamaño similar para todas las valoraciones?

## 2.-  Procesamiento de texto sobre las reviews
Eliminamos stopwords (TF-idf)
aplicamos métodos de lematizacion - TextBlob, NLTK
extraemos entidades (tokens)

## 3.- Extracción de muestras de test y de entrenamiento para el modelo
## 4.- Entrenamiento del modelo a partir de los datos de test.
## 5.- Ejecución del modelo. Evaluación de resultados, comparación con otros modelos Sentiment Analysis.

In [5]:
from textblob import TextBlob
from textblob import Word
import nltk
# nltk.download() hay que hacerlo la primera vez para cargar todos los corpus necesarios
#nltk.download()

## funcion que elimina las stopwords y aplica lematizacion

In [8]:
from nltk.corpus import stopwords

cachedStopWords = stopwords.words("english")
cachedStopWords


In [76]:
babyDf = pd.DataFrame([convert(line) for line in file('reviews_Baby_5.json')])
babyDf.to_csv(csv_filename, encoding='utf-8', index=False)


In [498]:
import string
def remove_punctuation(text):  
    #salida = text.translate(None, string.punctuation).lower()
    salida = text.translate(None, string.punctuation)
    return salida

def lemmatizeReviewText(row):
    words = TextBlob(row["review_clean"]).words.lemmatize()
    ext = ' '.join(word for word in words if word not in (cachedStopWords))
    return ext

In [477]:
babyDf = pd.DataFrame([convert(line) for line in file('reviews_Baby_5.json')])
# forzamos la columna reviewText como str para el correcto funcionamiento de remove_punctuation
babyDf['reviewText'] = babyDf['reviewText'].astype(str)
babyDf['review_clean'] = babyDf['reviewText'].apply(remove_punctuation)
babyDf['review_clean']= babyDf.apply(lemmatizeReviewText, axis=1)
babyDf['review_clean']=babyDf['review_clean'].apply(lambda x: x.lower())

babyDf.to_csv('reviews_Baby_5.csv', header=True,quoting=1)


In [599]:
# Tratamos el dataset de Pets
petsDf = pd.DataFrame([convert(line) for line in file('reviews_Pet_Supplies_5.json')])
# forzamos la columna reviewText como str para el correcto funcionamiento de remove_punctuation
petsDf['reviewText'] = petsDf['reviewText'].astype(str)
petsDf['review_clean'] = petsDf['reviewText'].apply(remove_punctuation)
petsDf['review_clean']= petsDf.apply(lemmatizeReviewText, axis=1)
petsDf['review_clean']= petsDf['review_clean'].apply(lambda x: x.lower())
petsDf.to_csv('reviews_Pet_Supplies_5.csv', header=True,quoting=1)

# Generamos el dataframe principalDf a partir de los dataframes  petsDf y babyDf

In [601]:


principalDf = [babyDf,petsDf]
babiesPetsDf = pd.concat(principalDf)
# renombramos la columna overall por review_overall
babiesPetsDf['review_overall']=babiesPetsDf['overall']
del(babiesDf['overall'])
len(babiesPetsDf)

318628

# Convertimos el dataFrame al csv que sera la fuente de entrada al modelo de ML

In [602]:
babiesPetsDf.to_csv('babies+pets_reviews.csv', header=True,quoting=1)

In [4]:
# comprobamos que está bien generado el csv volviendolo a cargar en el DF
import json
import pandas as pd
babiesPetsDf=pd.read_csv('babies+pets_reviews.csv',header=False,sep=',')
babiesPetsDf['review_clean'].sample(10)



268599    nothing great thing say blue buffalo product r...
110798    attend work conference thought would purchase ...
218728    love company quality product excellent custome...
220497    competitor product wa dark color crumbly made ...
314210    easy install look great color matching highlan...
66635     nice soft tie easily gentle teething son gum p...
73390     product worth super sturdy imagine going last ...
186168    isnt favorite good variety diet girl like pate...
191465    ok dog like favorite work toy use treat since ...
284166    beautiful vivid color sturdy even withheld 4 m...
Name: review_clean, dtype: object

In [12]:
#  crear un diccionario {palabra:ocurrencias}
import numpy as np
cnt = {}
for linea in babiesPetsDf['review_clean'].astype(str).values:
    for word in linea.split():
        if (word not in cnt):
            cnt[word] = 1
        else:
            cnt[word] += 1
            
wordsSerie=pd.Series(cnt,index=cnt.keys())
wordsSerie=wordsSerie[wordsSerie.values>1]
data={'a':wordsSerie.index,'b':wordsSerie.values}
wordsDf=pd.DataFrame(data=data, index=np.arange(len(wordsSerie)))
wordsDf=wordsDf.sort('b', ascending=False).head(1500)

# Generamos el array de palabras importantes

In [14]:
important_words = str([str(s) for s in wordsDf['a']])
f= open('important_words.json', 'w') 
f.write(important_words.replace("'",'"'))
f.close()
wordsDf.sample()

# Procesamiento de us_tweets.json --> us_tweets.csv
conversión de us_tweets_json a us_tweets.csv para aplicar proceso de lematización y eliminación de stopwords

In [741]:
# utilizado para descartar caracteres no ASCII
def is_ascii(s):
    return all(ord(c) < 128 for c in s)

# Devuelve un dataframe a partir del csv de reviews de Tweeter
def getTweetsDF(json_filename):
    csv_filename = '%s.csv' % json_filename[:-5]
    print 'Converting %s to %s' % (json_filename, csv_filename)
    df = pd.DataFrame( columns=['id','text','created_at','geo','city'])

    json_data=open(json_filename).read()
    ob = json.loads(json_data)
    results = ob['results']
    print(len(results))
    for k  in results:

        if (not k['geo']):
            geo = ''
        else:
            idd = k['id_str']
            #preguntamos si el texto está en ASCII para evitar que entren  caracteres UNICODE problematicos en NLP
            if (is_ascii( k['text'])):
                text = k['text']
            else:
                text = ''
            created_at = k['created_at']
            geo= ', '.join(map(str,k['geo']['coordinates']))
            if (is_ascii(k['place']['full_name'])):
                city = k['place']['full_name']
            else:
                city = ''
            SR_row = pd.Series({'id':idd, 'text':text, 'created_at':created_at,'geo':'['+geo+']','city':city},name=len(df))
            df=df.append(SR_row)
    return df

dfTweeter = getTweetsDF('USA_geo.json')

Converting USA_geo.json to USA_geo.csv
9354


# Tratamos el dataset de us_tweets

In [742]:
# lematiza y elimina Stop Words de la fila del dataset de reviews de Tweeter
def lemmatizeReviewTextForTweets(row):


    if (type(row['text']==str)):
        words = TextBlob(row['text']).words.lemmatize()
        ext = ' '.join(word for word in words if word not in (cachedStopWords))
    else:
        ext = ''
    return ext



In [743]:

dfTweeter['text_clean'] = dfTweeter.apply(lemmatizeReviewTextForTweets,axis=1)
dfTweeter['text_clean'] = dfTweeter['text_clean'].astype(str)
dfTweeter['text_clean'] = dfTweeter['text_clean'].apply(remove_punctuation)
# nos quedamos solo con los que tengan informado el campo text_clean
dfTweeter=dfTweeter[dfTweeter['text_clean']!='']

dfTweeter.to_csv('us_tweets.csv', header=True,quoting=1)

In [738]:
dfTweeter.to_csv('us_tweets.csv', header=True,quoting=1)

AttributeError: 'NoneType' object has no attribute 'to_csv'